<a id='08.1-bullet'></a> 
## 1. Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:
RANDOM_STATE = 123456

In [4]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('../datasets/insurance.csv')

In [5]:
display(data.head(1))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0


<a id='08.2-bullet'></a> 
## 2. Умножение матриц

Проверим изменится ли качество линейной регрессии, если умножить признаки на обратимую матрицу.

Рассчитаем параметры линейной регрессии.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$a=Xw$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<b>Нам нужно доказать что предсказания полученные при обучении на преобразованной матрице признаков равны предсказаниям полученным при обучении на исходных признаках.</b>

Предсказания по преобразованной матрице признаков:

$$a_p=X_pw_p$$

Где
- $X_p$ — преобразованная матрица признаков

- $w_p$ — вектор весов линейной регрессии, полученный с помощью обучения линейной регрессии на преобразованной матрице признаков.

<a id='ineer1-bullet'></a>
Необходимо доказать, что $a = a_p$, то есть $X w = X_p w_p$ или, так как $X_p = X P$:

$$w_p = P^{-1} w$$

Запишем формулу для весов для преобразованной матрицы:

$$w_p = ((XP)^T (XP))^{-1} (XP)^T y$$

Воспользуемся свойством что транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке, получим формулу:

$$w_p = ((P^T X^T) (X P))^{-1} P^T X^T y$$

Воспользуемся свойством обратных матриц, которое заключается в том, что матрица, обратная произведению матриц, равна произведению матриц, обратных матрицам-сомножителям, взятому в обратном порядке, получим формулу:

$$w_p = (X P)^{-1} (P^T X^T)^{-1} P^T X^T y$$

и ещё раз преобразуем формулу, воспользовавшись тем же свойством:

$$w_p = P^{-1} X^{-1} X^{T^{-1}} P^{T^{-1}} P^T X^T y$$

В нашей формуле получается что $P^{T-1} P^T = E$. А умножение на единичную матрицу не меняет её. Таким образом получается формула:

$$w_p = P^{-1} X^{-1} X^{T^{-1}} X^T y$$

Снова воспользуемся свойством обратных матриц, которое заключается в том, что матрица, обратная произведению матриц, равна произведению матриц, обратных матрицам-сомножителям, взятому в обратном порядке, получим формулу:

$$w_p = P^{-1} (X^T X)^{-1} X^T y$$

Так как для матриц хоть и необходимо строго соблюдать положение множителей, но порядок умножения не имеет значения, то можно записать формулу в следующем виде:

$$w_p = P^{-1} ((X^T X)^{-1} X^T y)$$

В скобках у нас получилось формула для вычисления весов обычной матрицы, таким образом получаем:

$$w_p = P^{-1} w$$

Что и требовалось доказать.

<b>Вывод:</b> Алгоритм преобразования исходных признаков с помощью обратимой матрицы не ухудшает качество предсказания линейной регрессии, что было доказано математически.

<a id='08.3-bullet'></a> 
## 3. Алгоритм преобразования

Напишем алгоритм преобразования признаков.

1. Сгенерируем рандомную обратимую матрицу с размером стороны равной количеству признаков.
2. Умножим матрицу с признаками на эту матрицу.
3. Преобразуем полученную матрицу в датафрейм с новым набором признаков.

В итоге алгоритм преобразования вернёт нам датафрейм с закодированными признаками.

<a id='08.4-bullet'></a> 
## 4. Проверка алгоритма

Напишем функцию для нахождения метрики R2 у линейной регрессии

In [6]:
def get_LinearRegression_R2(X, y):
    train_features, test_features, train_target, test_target = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)
    model = LinearRegression()
    model.fit(train_features, train_target)
    return model.score(test_features, test_target)

Функция создания случайной матрицы с количеством строк и столбцов как в исходной с попыткой её обратить пока это не удастся. 

In [7]:
def get_invert_matrix(size, random_state=RANDOM_STATE):
    random.seed(random_state)
    X = np.random.normal(size=size)
    try:
        return np.linalg.inv(X)
    except:
        return get_invert_matrix(size)

Функция кодирования

In [8]:
def get_coded_matrix(df, M):
    return pd.DataFrame(df.values @ M, columns=df.columns)

Данный алгоритм преобразовывает признаки с помощью случайной обратимой матрицы. Успешность применения данного алгоритма с точки зрения качества предсказания доказан в предыдущем пункте.

In [9]:
def transform_features(features):
    M = get_invert_matrix((features.shape[1], features.shape[1]))
    return get_coded_matrix(features, M)    

Проверим полученный алгоритм. Для этого сравним качество моделей до и после преобразования.

In [10]:
target = data[['Страховые выплаты']]
features = data.drop(['Страховые выплаты'], axis=1)

In [11]:
print('Качество линейной регрессии для исходных данных: R2 =', 
      get_LinearRegression_R2(features, target))

print('Качество линейной регрессии для закодированных данных: R2 =', 
      get_LinearRegression_R2(transform_features(features), target))

Качество линейной регрессии для исходных данных: R2 = 0.4192116037042861
Качество линейной регрессии для закодированных данных: R2 = 0.41921160370415145


Как видно из результата качество моделей одинаково вплоть до -цатого знака после запятой.

Таким образом, алгоритм кодирования данных считаю подобран и реализован успешно с сохранением качества моделей предсказания.